In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir(r'G:\main\data\Data Mining\College Experience Study Dataset\EMA')
data = pd.read_csv('general_ema.csv')
data_main = data[np.isnan(data['pam'])]
ind = list(data_main.index)
data_main = data.drop(index=ind,axis=0)
data_main = data_main.reset_index()
data_main = data_main.drop(['index'], axis=1)
EMA = data_main[['uid','day','pam', 'phq4_score', 'social_level', 'stress']]
describe = EMA.describe()
describe = describe.drop(['day'],axis=1)

In [ ]:
data_main

In [ ]:
plt.figure(dpi=80,figsize=(16,10))
plt.subplot(2,4,1)
x = ['pam', 'phq4', 'social', 'stress']
y = list(describe.loc['count', :])
plt.scatter(x,y)
plt.title('count', loc='left')

plt.subplot(2,4,2)
y = list(describe.loc['mean', :])
plt.scatter(x,y)
plt.title('mean', loc='left')

plt.subplot(2,4,3)
y = list(describe.loc['std', :])
plt.scatter(x,y)
plt.title('std', loc='left')

plt.subplot(2,4,4)
y = list(describe.loc['min', :])
plt.scatter(x,y)
plt.title('min', loc='left')

plt.subplot(2,4,5)
y = list(describe.loc['25%', :])
plt.scatter(x,y)
plt.title('25%', loc='left')

plt.subplot(2,4,6)
y = list(describe.loc['50%', :])
plt.scatter(x,y)
plt.title('50%', loc='left')

plt.subplot(2,4,7)
y = list(describe.loc['75%', :])
plt.scatter(x,y)
plt.title('75%', loc='left')

plt.subplot(2,4,8)
y = list(describe.loc['max', :])
plt.scatter(x,y)
plt.title('max', loc='left')

plt.show()

In [ ]:
plt.figure(dpi=80,figsize=(10,10))
plt.subplot(2,2,1)
plt.hist(list(EMA['pam']),10)
plt.title('pam')
plt.subplot(2,2,2)
plt.hist(list(EMA['phq4_score']),10)
plt.title('phq4')
plt.subplot(2,2,3)
plt.hist(list(EMA['social_level']),5)
plt.title('social')
plt.subplot(2,2,4)
plt.hist(list(EMA['stress']),5)
plt.title('stress')
plt.show()

In [ ]:
testee_pam_median = []
testee_phq4_median = []
testee_pam_std = []
testee_phq4_std = []
for x in set(EMA['uid']):
    testee = EMA[EMA['uid'] == x]
    testee = testee.describe()
    testee_pam_median.append(testee.loc['50%', 'pam'])
    testee_phq4_median.append(testee.loc['50%', 'phq4_score'])
    testee_pam_std.append(testee.loc['std', 'pam'])
    testee_phq4_std.append(testee.loc['std', 'phq4_score'])

plt.figure(dpi=80,figsize=(10,10))
plt.subplot(2,2,1)
plt.hist(testee_pam_median,10)
plt.title('pam median')
plt.subplot(2,2,2)
plt.hist(testee_phq4_median,10)
plt.title('phq4 median')
plt.subplot(2,2,3)
plt.hist(testee_pam_std,10)
plt.title('pam std')
plt.subplot(2,2,4)
plt.hist(testee_phq4_std,10)
plt.title('phq4 std')
plt.show()

做个时间序列展示：

In [ ]:
a = set(EMA['day'])
a = sorted(a)
a[-1]

单个样本在采样时间上存在很大的差异，进一步考虑在所有样本中共有的采样时间

In [ ]:
a = set(data['day'])
for x in set(data['uid']):
    testee = data[data['uid'] == x]
    b = set(testee['day'])
    a = set(list(a & b))
    print(a)
    print(b)

看的出来，采样时间也非常散乱，没能取到所有采样时间的交集

In [ ]:
import datetime
today_time =datetime.datetime.now()
def date_time(starttime,endtime):
    listx = []
    starttime = datetime.datetime.strptime(starttime, '%Y-%m-%d')
    endtime = datetime.datetime.strptime(endtime, '%Y-%m-%d')
    while starttime<endtime:
        select_time = starttime.strftime("%Y-%m-%d")
        select_time = str(select_time).replace('-','')
        listx.append(int(select_time))
        starttime+=datetime.timedelta(days=1)
    return listx
time_stamp = date_time(starttime='2017-09-07',endtime='2022-07-03')


PAM区间为0~16，phq4区间为0~12

In [ ]:
time_stamp[-1]

In [ ]:
x = list(range(len(time_stamp)))
plt.figure(dpi=300,figsize=(50,10))
plt.subplot(2,1,1)
plt.title('pam')
plt.xlim(0,len(time_stamp))
plt.ylim(0,16)
for y in range(len(EMA)):
    time = int(EMA.loc[y, 'day'])
    value = float(EMA.loc[y,'pam'])
    plt.scatter([time_stamp.index(time)],[value],c='r',)
plt.show()

考虑用数据均值和标准差作图：

In [ ]:
EMA

In [ ]:
dicts = {}
for x in range(len(EMA)):
    if EMA.loc[x,'day'] in dicts.keys():
        dicts[EMA.loc[x,'day']].append(EMA.loc[x,'pam'])
    else:
        dicts[EMA.loc[x,'day']] = []
        dicts[EMA.loc[x,'day']].append(EMA.loc[x,'pam'])
for x in dicts.keys():
    para = dicts[x]
    dicts[x] = [np.mean(para), np.std(para,ddof=1)]

dict2 = {}
for x in range(len(EMA)):
    if EMA.loc[x,'day'] in dict2.keys():
        dict2[EMA.loc[x,'day']].append(EMA.loc[x,'phq4_score'])
    else:
        dict2[EMA.loc[x,'day']] = []
        dict2[EMA.loc[x,'day']].append(EMA.loc[x,'phq4_score'])
for x in dict2.keys():
    para = dict2[x]
    dict2[x] = [np.mean(para), np.std(para,ddof=1)]

In [ ]:
dicts
y = list(dicts.keys())
y = sorted(y)
plt.figure(dpi=300,figsize=(40,30))
plt.subplot(2,1,1)
plt.title('pam')
plt.xlim(0,len(y))
plt.ylim(0,20)
for z in dicts.keys():
    plt.scatter([y.index(z)],[dicts[z][0]],c='c')
    plt.scatter([y.index(z)],[dicts[z][0] + dicts[z][1]],c='m')
    plt.scatter([y.index(z)],[dicts[z][0] - dicts[z][1]],c='y')

plt.subplot(2,1,2)
plt.title('phq4')
plt.xlim(0,len(y))
plt.ylim(-5,15)
for z in dict2.keys():
    plt.scatter([y.index(z)],[dict2[z][0]],c='c')
    plt.scatter([y.index(z)],[dict2[z][0] + dict2[z][1]],c='m')
    plt.scatter([y.index(z)],[dict2[z][0] - dict2[z][1]],c='y')
plt.show()

对所有的情绪量表参数都做以上处理以及可视化！

In [ ]:
import sys
from importlib import reload
sys.path.insert(0,r'G:\main\data\Data Mining\College Experience Study Dataset\EMA')

import emotion_visualization
reload(emotion_visualization)


In [ ]:
name_list = list(data_main.columns)
name_list.remove('uid')
name_list.remove('day')
for x in name_list:
    emotion_visualization.emotion_visualization(x)